# Lab 1 - Set covering
Given an integer `N`, and a list of lists containing random integers $\in[0, N-1]$, find a combination of lists, $L_c$ consisting of all integers $n \ \in[0, N-1]$ minimizing the number of elements in $L_c$.

In [ ]:
# given function to yield list of lists
import random

def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [ ]:
N = 5
seed = 42
l = problem(N, seed)

In [27]:
random.randint(10//5,10//2)

4

In [32]:
# imports
import logging
import numpy as np
import graphviz as gz
from typing import Callable

In [29]:
# logging
logging.basicConfig(format="%(message)s", level=logging.INFO)

### Acknowledgement
Re-use code from [Squillero's github](https://github.com/squillero/computational-intelligence/blob/master/2022-23/8-puzzle.ipynb)

In [37]:
# define state-class
class State:
    def __init__(self, data: list):
        self._data = data.copy()
        #self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [33]:
# search function
def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    while state is not None and not goal_test(state):
        for a in possible_actions(state):
            new_state = result(state, a)
            cost = unit_cost(a)
            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    return list(reversed(path))

In [51]:
# goal state
GOAL = State(list(range(N)))
logging.info(f"Goal:\n{GOAL}")


def goal_test(state):
    return all(elem in state.data for elem in GOAL.data)

Goal:
[0, 1, 2, 3, 4]


In [53]:
l  = [1,3,2,1,1,0,4]
goal_test(State(l))

True

In [44]:
all(elem in GOAL.data for elem in l)

True

In [42]:
GOAL.data

[0, 1, 2, 3, 4]